# Homework - Production

## Question 1
- Install BentoML
- What's the version of BentoML you installed?

In [1]:
!bentoml --version

bentoml, version 1.0.7


## Question 2
Run the notebook which contains random forest model from module 6 i.e previous module and save the model with BentoML. To make it easier for you we have prepared this notebook.

How big approximately is the saved BentoML model? Size can slightly vary depending on your local development environment. Choose the size closest to your model.

- 924kb
- 724kb
- 114kb
- 8kb

In [2]:
!stat ~/bentoml/models/credit_risk_model/w373v2sqpwrjoaav/saved_model.ubj -c %s

201435


## Question 3
Say you have the following data that you're sending to your service:

```json
{
  "name": "Tim",
  "age": 37,
  "country": "US",
  "rating": 3.14
}
```
What would the pydantic class look like? You can name the class `UserProfile`.

In [3]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    name: str
    age: int
    country: str
    rating: float

## Question 4
We've prepared a model for you that you can import using:

```bash
curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel.bentomodel
bentoml models import coolmodel.bentomodel
```
What version of scikit-learn was this model trained with?

- 1.1.1
- 1.1.2
- 1.1.3
- 1.1.4
- 1.1.5

In [4]:
!curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel.bentomodel

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1724  100  1724    0     0   2218      0 --:--:-- --:--:-- --:--:--  2218


In [5]:
!bentoml models import coolmodel.bentomodel

Model(tag="mlzoomcamp_homework:qtzdz3slg6mwwdu5") imported


In [6]:
!cat ~/bentoml/models/mlzoomcamp_homework/qtzdz3slg6mwwdu5/model.yaml

name: mlzoomcamp_homework
version: qtzdz3slg6mwwdu5
module: bentoml.sklearn
labels: {}
options: {}
metadata: {}
context:
  framework_name: sklearn
  framework_versions:
    scikit-learn: 1.1.1
  bentoml_version: 1.0.7
  python_version: 3.9.12
signatures:
  predict:
    batchable: false
api_version: v1
creation_time: '2022-10-13T20:42:14.411084+00:00'


## Question 5
Create a bento out of this scikit-learn model. The output type for this endpoint should be `NumpyNdarray()`

Send this array to the Bento:

```[[6.4,3.5,4.5,1.2]]```

You can use curl or the Swagger UI. What value does it return?

- 0
- 1
- 2
- 3

(Make sure your environment has Scikit-Learn installed)

In [7]:
!pip show -V scikit-learn

Name: scikit-learn
Version: 1.1.1
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /home/rafaelsguerra/.local/share/virtualenvs/07_-_production-h-uEbiEX/lib/python3.8/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: 


In [8]:
# After running bentoml serve bento_service.py:service in the terminal:

!curl -X 'POST' \
  'http://localhost:3000/predict' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '[[6.4,3.5,4.5,1.2]]'

[1]

## Question 6
Ensure to serve your bento with --production for this question

Install locust using:

```bash
pip install locust
```

Use the following locust file: locustfile.py

Ensure that it is pointed at your bento's endpoint (In case you didn't name your endpoint "classify")

Configure 100 users with ramp time of 10 users per second. Click "Start Swarming" and ensure that it is working.

Now download a second model with this command:

```bash
curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel
```

Or you can download with this link as well: https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel

Now import the model:

```bash
bentoml models import coolmodel2.bentomodel
```

Update your bento's runner tag and test with both models. Which model allows more traffic (more throughput) as you ramp up the traffic?

**Hint 1**: Remember to turn off and turn on your bento service between changing the model tag. Use Ctl-C to close the service in between trials.

**Hint 2**: Increase the number of concurrent users to see which one has higher throughput

Which model has better performance at higher volumes?

- The first model
- The second model

In [9]:
!curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel2.bentomodel

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1728  100  1728    0     0   2437      0 --:--:-- --:--:-- --:--:--  2437


In [10]:
!bentoml models import coolmodel2.bentomodel

Model(tag="mlzoomcamp_homework:jsi67fslz6txydu5") imported


The images below represent the load test made with locust. The left chart corresponds to the first model, while the right chart corresponds to the second model

Requests per second:

![requests_per_second](./assets/rps.png)

Response times

![response_times](./assets/rt.png)

Total users

![total_users](./assets/users.png)

Although the response times got a bit higher with more users accessing the endpoint, the second model was able to receive more requests than the first model when both had 500 users and the slowest requests didn't get past 1 second. So the second model has better peformance than the first.